In [1]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

# Recreate model architecture exactly as in training notebook
class FinbertBackbone(nn.Module):
    def __init__(self, modelName: str = "ProsusAI/finbert"):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(modelName)
        self.hiddenSize = self.encoder.config.hidden_size  # 768 for BERT-base

    def forward(self, input_ids, attention_mask):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        cls = out.last_hidden_state[:, 0]  # [CLS] token
        return cls  # [batch, hidden]

class BinaryHead(nn.Module):
    def __init__(self, inFeatures: int, pDrop: float = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(pDrop)
        self.fc = nn.Linear(inFeatures, 1)  # single logit

    def forward(self, x):
        x = self.dropout(x)
        logits = self.fc(x).squeeze(-1)    # [batch]
        return logits

class FinbertBinaryClf(nn.Module):
    def __init__(self, modelName: str = "ProsusAI/finbert", pDrop: float = 0.1):
        super().__init__()
        self.backbone = FinbertBackbone(modelName)
        self.head = BinaryHead(self.backbone.hiddenSize, pDrop)

    def forward(self, input_ids, attention_mask):
        feats = self.backbone(input_ids, attention_mask)
        logits = self.head(feats)
        return logits


d:\CODE\Deep_ml_project\AI-financial-sentiment-prediction\venv\Lib\site-packages\transformers\utils\hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
# Load tokenizer and model weights
model_name = "ProsusAI/finbert"

# Choose which model to use:
# Option 1: Finetuned model (higher F1, trained with 2-phase approach)
weights_path = "finbert_finetuned.pt"  

# Option 2: Frozen head only (if finbert_finetuned.pt doesn't exist yet)
# weights_path = "finbert_custom_head.pt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name)

# IMPORTANT: Dropout must match training
# For finbert_finetuned.pt: use pDrop=0.1 (original config)
# For finbert_custom_head.pt (if using old frozen model): use pDrop=0.3
model = FinbertBinaryClf(modelName=model_name, pDrop=0.1).to(device)

# Load checkpoint (saved as dictionary with 'model_state_dict' key)
checkpoint = torch.load(weights_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Device: {device}")
print(f"Model loaded: {weights_path}")
print(f"  Total epoch: {checkpoint['epoch']+1}")
print(f"  Validation F1: {checkpoint['val_f1']:.3f}")
print(f"  Validation Loss: {checkpoint['val_loss']:.4f}")


Device: cuda
Model loaded from epoch: 18
Validation F1: 0.736
Validation Loss: 0.5569


In [3]:
import re

def clean_text(text):
    """Clean tweet text (same as training preprocessing)"""
    text = str(text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"^user:\s*", "", text, flags=re.IGNORECASE)
    text = re.sub(r"^user\s*", "", text, flags=re.IGNORECASE)
    text = re.sub(r"[\"]+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

@torch.no_grad()
def predict_texts(texts, tokenizer, model, max_len=128, threshold=0.5, device=None):
    """Predict sentiment for input texts
    
    Returns:
        labels: 1=positive, 0=negative
        probs: probability of positive class
    """
    device = device or next(model.parameters()).device
    model.eval()
    
    if isinstance(texts, str):
        texts = [texts]
    
    # Clean texts (important!)
    texts = [clean_text(t) for t in texts]
    
    enc = tokenizer(
        texts, truncation=True, padding="max_length", max_length=max_len, return_tensors="pt"
    )
    logits = model(enc["input_ids"].to(device), enc["attention_mask"].to(device))
    probs = torch.sigmoid(logits).cpu().numpy()
    labels = (probs >= threshold).astype(int)  # 1=positive, 0=negative
    return labels, probs


In [4]:
# Enter your text/tweet here
text = "Great earnings beat and strong guidance!"

labels, probs = predict_texts(text, tokenizer, model, max_len=128, threshold=0.5, device=device)
label = int(labels[0])
prob = float(probs[0])

sentiment_str = "positive - 1" if label == 1 else "negative - 0"
print(f"Input: {text}")
print(f"Prediction: {sentiment_str}")
print(f"Positive probability: {prob:.4f}")


Input: Great earnings beat and strong guidance!
Prediction: positive - 1
Positive probability: 0.9461


In [5]:
# Optional: interactive prediction
while True:
    try:
        user_text = input("Enter a tweet/text to analyze: ")
        if user_text == "":
            break
        if user_text.strip():
            labels, probs = predict_texts(user_text, tokenizer, model, max_len=128, threshold=0.5, device=device)
            label = int(labels[0])
            prob = float(probs[0])
            sentiment_str = "positive - 1" if label == 1 else "negative - 0"
            print(f"input: {user_text}")
            print(f"Prediction: {sentiment_str}")
            print(f"Positive probability: {prob:.4f}")
        else:
            print("No text provided.")
    except EOFError:
        # In non-interactive environments, input() may fail
        pass


input: i will go to the stock market and buy me some tesla stock
Prediction: positive - 1
Positive probability: 0.6244
input: elon musk is my hero
Prediction: positive - 1
Positive probability: 0.6803
input: i love tesla
Prediction: positive - 1
Positive probability: 0.7191
input: tesla is great
Prediction: positive - 1
Positive probability: 0.7690
input: tesla is bad
Prediction: positive - 1
Positive probability: 0.5591
input: i hate tesla
Prediction: negative - 0
Positive probability: 0.0995
input: god i hate this
Prediction: negative - 0
Positive probability: 0.1332
input: tesla is bad
Prediction: positive - 1
Positive probability: 0.5591
input: tesla is the worst company ever
Prediction: negative - 0
Positive probability: 0.0328
input: its so bad i want to fly
Prediction: negative - 0
Positive probability: 0.2086
input: microsoft is bad
Prediction: negative - 0
Positive probability: 0.4112
input: i just sold all my stocks
Prediction: negative - 0
Positive probability: 0.3469
input: